In [5]:
%cd /content/drive/My\ Drive/DataExploration
%ls

/content/drive/My Drive/DataExploration
 bellevue_almshouse_current_09_22_17.csv
 covidTweet/
 Dashboard.ipynb
 DataExploration.ipynb
 innercity.csv
 medium_data_2019_01_06
 mushrooms.csv
'Public Script Sources - public_scripts.csv'
 raw_script_urls.txt
'Uber Request Data.csv'
'Womens Clothing E-Commerce Reviews.csv'
 zomato/


# Covid Tweet
https://towardsdatascience.com/creating-interactive-jupyter-notebooks-and-deployment-on-heroku-using-voila-aa1c115981ca

In [30]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import itertools, string

from ipywidgets import interact, fixed
import ipywidgets as widgets

from textblob import TextBlob
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from PIL import Image

In [6]:
df = pd.read_csv('covidTweet/tweet_covid.csv', parse_dates=True, index_col='created_at')
df

,hashtags,favorite_count,place,text,user_screen_name,user_description,user_followers_count,user_name,cleaned_text,text_sentiment
created_at,,,,,,,,,,
2020-03-25 11:50:02+05:30,NaN,0,"Pune, India",Meditation and mindfulness tips for coping wit...,ShalomFloat,Shalom float spa is the first dedicated float ...,382,Shalom Float Spa,meditation mindfulness tips coping corona viru...,Negative
2020-03-25 12:06:25+05:30,NaN,0,"Gandhinagar, India",Contribute to INDIA FIGHTS CORONA @ Gandhinaga...,prakashkalal,NaN,175,prks7,contribute india fights corona gandhinagar guj...,Neutral
2020-03-25 11:48:32+05:30,NaN,0,"Bidhan Nagar, India",@DelhiPolice @DCPEastDelhi @msisodia @AamAadmi...,Bhartendulkar,Senior Media Analyst @Indianpac | Ex Sr Report...,609,Bhartendu Sharma,delhipolice dcpeastdelhi msisodia aamaadmipart...,Neutral
2020-03-25 11:35:46+05:30,SSC_UFM_MAT_KARONA Corona UFM,9,"Maharashtra, India",#SSC_UFM_MAT_KARONA\n#Corona is slow poison #U...,patilkiii,"Ab khushi de ke aazma le khuda, In ghamon se t...",259,ùï¨ùñáùñçùñéùñëùñÜùñòùñç ùïæùñöùñì...,sscufmmatkaronacorona slow poison ufm instant ...,Positive
2020-03-25 12:01:10+05:30,Corona pritamkumarmurari Voice,0,"Bokaro Steel City, India",https://t.co/P7eQiq8tFa Mata Di \nDoston ise #...,SINGERMurariPa1,"Singing,Dancing,Acting,Entertainment",2,SINGER=Murari Pathak,mata di doston ise corona ke tarah faila bhai ...,Neutral
...,...,...,...,...,...,...,...,...,...,...
2020-05-07 09:16:30+05:30,HardAreaAllowance HCWs Corona Doctors Covid Ha...,10,"New Delhi, India",#HardAreaAllowance for #HCWs: During #Corona p...,ompsychiatrist,"Psychiatrist, Avid Reader, Thinker",3241,Dr Om Prakash MD (Psychiatry),hardareaallowance hcws corona pandemic healthc...,Neutral
2020-05-07 08:27:32+05:30,NaN,0,"Faridabad, India",@ZeeNews @sudhirchaudhary But who did not had ...,Dubeymkd1944,NaN,21,Satyamew Jayte,zeenews sudhirchaudhary corona illness u depri...,Negative
2020-05-07 09:17:42+05:30,NaN,5,"Orissa, India",No Surprises https://t.co/6RWca4U7mP,Bikram_Odisha,"A learner. Fan of @Naveen_Odisha & @iamsrk , p...",1937,Bikram Swain,surprises,Neutral


## Distribution

In [7]:
# Overall WeekDays Distribution

ana1 = df.index.day_name().value_counts()
days = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
values = tuple(ana1[i] for i in days)

fig = px.bar(ana1, x=days, y=values, color=values,
             labels={'x': 'Day of Week', 'color':'Number of Tweets'},
             template='plotly_dark', text=values, color_continuous_scale='Rainbow',
            )
fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                  yaxis={'visible': False, 'showticklabels': False},                  
                 )
fig.show()

In [8]:
# Monthly Distribution

ana2 = df.index.month_name().value_counts()

fig = go.Figure(data=[go.Pie(labels=ana2.index, values=ana2.values,
                             textinfo='label+percent', pull=[0.2, 0, 0],
                             title='Contribution of Tweets from each Month'
                             )])
fig.update_traces(hoverinfo='label+value',  textfont_size=15, 
                  marker=dict(line=dict(color='#eff542', width=2)), showlegend=False)

fig.update_layout(template='plotly_dark', font=dict(size=15))
fig.show()

In [9]:
# March Day Distribution

ana3 = df[df.index.month == 3].index.day.value_counts()

fig = px.bar(ana3, x=ana3.index, y=ana3.values, color=ana3.values,
             labels={'index': "March Day's", 'color':'Number of Tweets'},
             template='plotly_dark', text=ana3.values, color_continuous_scale='Rainbow',
            )
fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                  yaxis={'visible': False, 'showticklabels': False},
                 )
fig.show()

In [10]:
# April Day Distribution

ana4 = df[df.index.month == 4].index.day.value_counts()

fig = px.bar(ana4, x=ana4.index, y=ana4.values, color=ana4.values,
             labels={'index': "April Day's", 'color':'Number of Tweets'},
             template='plotly_dark', text=ana4.values,
             color_continuous_scale='Rainbow',
            )
fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                  yaxis={'visible': False, 'showticklabels': False},                  
                 )
fig.update_layout(hovermode="x")
fig.show()

In [11]:
# May Day Distribution

ana5 = df[df.index.month == 5].index.day.value_counts()

fig = px.bar(ana5, x=ana5.index, y=ana5.values, color=ana5.values,
             labels={'index': "May Day's", 'color':'Number of Tweets'},
             template='plotly_dark', text=ana5.values, 
             color_continuous_scale='Rainbow',
            )
fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                  yaxis={'visible': False, 'showticklabels': False},
                 )
fig.show()

In [12]:
# Hourly Distribution

ana6 = df.index.hour.value_counts()
fig = go.Figure()
fig.add_trace(go.Scatter(x=ana6.index,  y=ana6.values, mode='markers',
                         marker=dict(size=20, line_width=2,color=ana6.values,
                             colorscale='Rainbow',showscale=True)
                    ))
fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                  yaxis={'visible': True, 'showticklabels': False},
                  xaxis_title='Hour of Day (24-hour clock)',
                  yaxis_title='Number of Tweets', template='plotly_dark', 
                 )
fig.update_layout(hovermode="x")

In [13]:
# Top Hashtags in Tweets

ana7 = df[~df.hashtags.isnull()]
data = {}
allHashTags = tuple(itertools.chain.from_iterable(ana7.hashtags.str.lower().str.split().values.tolist()))

for i in allHashTags:
  if i not in data:        data[i] = 1
  else:        data[i] += 1
        
sortedHashTags = tuple(sorted(data.items(), key= lambda x: x[1])[::-1])

hashTagsDf = pd.DataFrame(sortedHashTags, columns=['hashTag', 'count'])

def displayHashTags(Top):
    fig = px.scatter(hashTagsDf[:Top], x="hashTag", y="count",
                     size="count", color="count", size_max=60,
                     labels={'count': ''}, color_continuous_scale='Rainbow',
                    )    
    fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                      yaxis={'visible': True, 'showticklabels': False},
                      xaxis_title='(#) Hash Tags', yaxis_title='Number of Tweets',
                      template='plotly_dark',                      
                     )    
    fig.show();

interact(displayHashTags, Top=widgets.IntSlider(min=0, max=30, step=1, value=10));

interactive(children=(IntSlider(value=10, description='Top', max=30), Output()), _dom_classes=('widget-interac…

In [14]:
# Top Places From Where Tweets were made

ana8 = pd.DataFrame(data=[df.place.value_counts().index, df.place.value_counts().values]).T
ana8 = ana8.rename(columns={0: "place", 1: 'count'})

def displayPlaces(Top):

    fig = px.bar(ana8[:Top], x='place', y='count', color='count',
                 labels={'place': "Location of Tweet", 'count':'Number of Tweets'},
                 template='plotly_dark', text='count', color_continuous_scale='Turbo',
                )
    fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                      yaxis={'visible': True, 'showticklabels': False, 'showgrid': False},                      
                     )    
    fig.update_traces(showlegend=False)

    fig.show()

interact(displayPlaces, Top=widgets.IntSlider(min=0, max=30, step=1, value=10));

interactive(children=(IntSlider(value=10, description='Top', max=30), Output()), _dom_classes=('widget-interac…

In [15]:
places_with_cordinates = pd.read_json('covidTweet/India_map_coordinates.json')

filtered_locations = places_with_cordinates[places_with_cordinates.map_locations != 'Not Found']

coordinates = tuple(zip(*filtered_locations.map_locations))

coordinates_lat = tuple(float(i) for i in coordinates[0])
coordinates_long = tuple(float(i) for i in coordinates[1])

filtered_locations = filtered_locations.rename(columns={'0': 'Place', '1': 'Tweet Count'}) 

fig = px.scatter_mapbox(filtered_locations,
                        lat=coordinates_lat, lon=coordinates_long,
                        hover_name="Place", hover_data=["Tweet Count"],
                        color_continuous_scale="Rainbow",
                        color=filtered_locations["Tweet Count"],
                        size=filtered_locations["Tweet Count"],
                        size_max=50, zoom=4, height=600, template='plotly_dark',
                       )
fig.update_layout(mapbox_style="carto-darkmatter",)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [17]:
# Most Tagged Person in Tweets

taggedPersons = df.text.str.findall(r"@[\w]+").values

mergedListPersons = tuple(itertools.chain.from_iterable([i for i in taggedPersons if len(i)!=0]))

tagsCount = {}

for i in mergedListPersons:
  if i not in tagsCount:        tagsCount[i] = 1
  else:        tagsCount[i] += 1

ana9 = pd.DataFrame(sorted(tagsCount.items(), key=lambda x: x[1])[::-1], columns=['Person', 'Count'])

def displayMostTagged(Top):

    fig = px.bar(ana9[:Top], x='Person', y='Count', color='Count',
                 labels={'Count':'Number of Tweets'},
                 template='plotly_dark', text='Count',
                 color_continuous_scale='tropic',
                )
    fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                      yaxis={'visible': False, 'showticklabels': False},                      
                     )    
    fig.update_traces(showlegend=False)

    fig.show();

interact(displayMostTagged, Top=widgets.IntSlider(min=0, max=30, step=1, value=10));

interactive(children=(IntSlider(value=10, description='Top', max=30), Output()), _dom_classes=('widget-interac…

In [18]:
# Trending Hashtags of Every Place

ana10 = df[['hashtags', 'place']]
ana10 = ana10[~ana10.hashtags.isnull()]

placesTags = ana10.groupby('place').sum()

def splitCountTags(row):
  tags = row.split()
  counts = {}
  for i in tags:
      if i not in counts:            counts[i] = 1
      else:            counts[i] += 1
  counts = sorted(counts.items(), key=lambda x: x[1])[::-1]
  return counts

placesTagsWithCount = placesTags.hashtags.apply(splitCountTags)
dropDownLocation = placesTagsWithCount.index

def displayLocationTags(place, count):
    
    temp = pd.DataFrame(placesTagsWithCount[place], columns=['Hash Tags', 'Count'])[:count]
    fig = px.bar(temp, x='Hash Tags', y='Count', color='Count',
                 title = f"{place}'s Trending Hash Tags",
                 #labels={'place': "Location of Tweet", 'count':'Number of Tweets'},
                 template='plotly_dark', text='Count',
                 color_continuous_scale='tropic',
                )

    fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                      yaxis={'visible': True, 'showticklabels': False, 'showgrid': False},                      
                     )
    fig.update_traces(showlegend=False)

    fig.show();

interact(displayLocationTags, place=dropDownLocation, count=widgets.IntSlider(min=0, max=30, step=1, value=10));

interactive(children=(Dropdown(description='place', options=('26 Block, west patel nagar', 'AAIMS', 'AIIMS', '…

In [19]:
#Most Engaged User on Twitter

ana11 = pd.DataFrame(df.user_screen_name.value_counts())
ana11 = ana11.rename(columns={'user_screen_name': 'Tweets Count'})

def displayMostEngagedUsers(Top):

    fig = px.bar(ana11[:Top], x=ana11[:Top].index, y='Tweets Count',
                 color='Tweets Count', labels={'x':'Twitter Handle'},
                 template='plotly_dark', text='Tweets Count',
                 color_continuous_scale='Rainbow',
                )
    fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                      yaxis={'visible': False, 'showticklabels': False},                      
                     )    
    fig.update_traces(showlegend=False)

    fig.show();

interact(displayMostEngagedUsers, Top=widgets.IntSlider(min=0, max=30, step=1, value=10));

interactive(children=(IntSlider(value=10, description='Top', max=30), Output()), _dom_classes=('widget-interac…

## Word cloud

In [20]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
stopWordsTofollow = set(stopwords.words('english'))

def removePunctuation(row):
    cleaned = ''.join([w for w in row if w not in string.punctuation])
    return cleaned

def removeExtras(column):
    patterns = ['@[\w+]', '#[\w+]', 'http\S+', '\n', '@']
    for pattern in patterns:
        column = column.str.replace(pattern, '')
    return column.str.lower()

def tokenizeSentence(row):
    return word_tokenize(row)

def removeStopWords(row):
    cleanedSentence = [w for w in row if w not in stopWordsTofollow]
    return ' '.join(cleanedSentence)

In [22]:
userFiltered = df.copy()
userFiltered = userFiltered[~userFiltered.user_description.isnull()]
userFiltered['cleaned_user_description'] = removeExtras(userFiltered.user_description)
userFiltered['cleaned_user_description'] = userFiltered.cleaned_user_description.apply(tokenizeSentence)
userFiltered['cleaned_user_description'] = userFiltered.cleaned_user_description.apply(removeStopWords)
userFiltered

,hashtags,favorite_count,place,text,user_screen_name,user_description,user_followers_count,user_name,cleaned_text,text_sentiment,cleaned_user_description
created_at,,,,,,,,,,,
2020-03-25 11:50:02+05:30,NaN,0,"Pune, India",Meditation and mindfulness tips for coping wit...,ShalomFloat,Shalom float spa is the first dedicated float ...,382,Shalom Float Spa,meditation mindfulness tips coping corona viru...,Negative,shalom float spa first dedicated float centre ...
2020-03-25 11:48:32+05:30,NaN,0,"Bidhan Nagar, India",@DelhiPolice @DCPEastDelhi @msisodia @AamAadmi...,Bhartendulkar,Senior Media Analyst @Indianpac | Ex Sr Report...,609,Bhartendu Sharma,delhipolice dcpeastdelhi msisodia aamaadmipart...,Neutral,senior media analyst ndianpac | ex sr reporter...
2020-03-25 11:35:46+05:30,SSC_UFM_MAT_KARONA Corona UFM,9,"Maharashtra, India",#SSC_UFM_MAT_KARONA\n#Corona is slow poison #U...,patilkiii,"Ab khushi de ke aazma le khuda, In ghamon se t...",259,ùï¨ùñáùñçùñéùñëùñÜùñòùñç ùïæùñöùñì...,sscufmmatkaronacorona slow poison ufm instant ...,Positive,"ab khushi de ke aazma le khuda , ghamon se toh..."
2020-03-25 12:01:10+05:30,Corona pritamkumarmurari Voice,0,"Bokaro Steel City, India",https://t.co/P7eQiq8tFa Mata Di \nDoston ise #...,SINGERMurariPa1,"Singing,Dancing,Acting,Entertainment",2,SINGER=Murari Pathak,mata di doston ise corona ke tarah faila bhai ...,Neutral,"singing , dancing , acting , entertainment"
2020-03-25 12:05:29+05:30,NaN,0,"Mumbai, India","@funnyhammy @GauravPandhi Dumbo, it's the amou...",amjadmaruf,A proud Indian.,3851,Amjad Maruf,funnyhammy gauravpandhi dumbo amount counts to...,Neutral,proud indian .
...,...,...,...,...,...,...,...,...,...,...,...
2020-05-07 09:07:13+05:30,Kerela India pinarayivijayan COVID19 FightAgai...,0,"Kolkata, India",Oh great indeed #Kerela #India #pinarayivijaya...,learnwithrohanb,Tutor | Web Developer |\nEvent Planner | Digit...,41,Rohan Bhattacharya,oh great indeed kerela india pinarayivijayan c...,Positive,tutor | web developer |event planner | digital...
2020-05-07 09:16:30+05:30,HardAreaAllowance HCWs Corona Doctors Covid Ha...,10,"New Delhi, India",#HardAreaAllowance for #HCWs: During #Corona p...,ompsychiatrist,"Psychiatrist, Avid Reader, Thinker",3241,Dr Om Prakash MD (Psychiatry),hardareaallowance hcws corona pandemic healthc...,Neutral,"psychiatrist , avid reader , thinker"
2020-05-07 09:17:42+05:30,NaN,5,"Orissa, India",No Surprises https://t.co/6RWca4U7mP,Bikram_Odisha,"A learner. Fan of @Naveen_Odisha & @iamsrk , p...",1937,Bikram Swain,surprises,Neutral,"learner . fan aveen_odisha & amsrk , passionat..."


In [31]:
corpusText = ''.join(df.cleaned_text.astype(str).values.tolist())
corpusUser = ' '.join(userFiltered.cleaned_user_description.values.tolist())

def generateWordCloud(Masking, corpus):
    
    mask = np.array(Image.open(f'./covidTweet/{Masking}.png'))
    wc = WordCloud(width=800, height=500, random_state=0, colormap='tab20c',  
               mask=mask, contour_width=1, stopwords=['amp'],
               contour_color='orange').generate_from_text(corpus)
    plt.figure(figsize=(12, 7))
    plt.imshow(wc, interpolation="bilinear")
    plt.style.use('dark_background')
    plt.axis('off')
    plt.show()

interact(generateWordCloud, Masking=['Modiji', 'Trump', 'India Map'], corpus=[('All Tweets', corpusText), ('User Description', corpusUser)]);

interactive(children=(Dropdown(description='Masking', options=('Modiji', 'Trump', 'India Map'), value='Modiji'…

In [32]:
# Sentiments Over Weeks and Days

ana12 = df.text_sentiment.value_counts()

fig = go.Figure(data=[go.Pie(labels=ana12.index, values=ana12.values,
                             textinfo='label+percent', pull=[0, 0, 0.2],
                             title='Total Sentiments in all Tweets'
                             )])
fig.update_traces(hoverinfo='label+value',  textfont_size=15, 
                  marker=dict(line=dict(color='#eff542', width=2)), showlegend=False)
fig.update_layout(template='plotly_dark',
                  font=dict(size=15))
fig.show();

In [33]:
positive = df[df.text_sentiment == 'Positive']
negative = df[df.text_sentiment == 'Negative']
neutral = df[df.text_sentiment == 'Neutral']

temp1 = positive.groupby(positive.index.day_name()).count()['text_sentiment']
temp2 = negative.groupby(negative.index.day_name()).count()['text_sentiment']
temp3 = neutral.groupby(neutral.index.day_name()).count()['text_sentiment']

days = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
values_pos = tuple(temp1[i] for i in days)
values_neg = tuple(temp2[i] for i in days)
values_neu = tuple(temp3[i] for i in days)

In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=days, y=values_pos,mode='lines', name='Positive Sentiment'
                    ))
fig.add_trace(go.Scatter(x=days, y=values_neg, mode='lines', name='Negative Sentiment'
                    ))
fig.add_trace(go.Scatter(x=days, y=values_neu, mode='lines', name='Neutral Sentiment'
                    ))
fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                  yaxis={'visible': True, 'showticklabels': True, 'showgrid': False},
                  xaxis_title='Week Days', yaxis_title='Number of Tweets',
                  template='plotly_dark',  title='Sentiment over Week'
                 )

In [35]:
temp4 = positive.groupby(positive.index.day).count()['text_sentiment']
temp5 = negative.groupby(negative.index.day).count()['text_sentiment']
temp6 = neutral.groupby(neutral.index.day).count()['text_sentiment']

fig = go.Figure()
fig.add_trace(go.Scatter(x=temp4.index, y=temp4.values, mode='lines',
                         name='Positive Sentiment'
                    ))
fig.add_trace(go.Scatter(x=temp5.index, y=temp5.values,mode='lines',
                         name='Negative Sentiment'
                    ))
fig.add_trace(go.Scatter(x=temp6.index, y=temp6.values, mode='lines',
                         name='Neutral Sentiment'
                    ))
fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                  yaxis={'visible': True, 'showticklabels': True, 'showgrid': False},
                  xaxis_title='Days', yaxis_title='Number of Tweets',
                  template='plotly_dark',  title='Sentiment over Days'
                 )
fig.update_layout(hovermode="x")

In [36]:
# Tweets Made by Huge Follower Handles

celebrities = df.groupby('user_screen_name').max('user_followers_count').sort_values(by='user_followers_count', ascending=False)

def displayCelebrities(Top):

  fig = px.bar(celebrities[:Top], x=celebrities[:Top].index, y='user_followers_count',
                color='user_followers_count',
                labels={'x':'Twitter Handle', 'user_followers_count': 'Followers'},
                template='plotly_dark', text='user_followers_count',
                color_continuous_scale='Rainbow',
              )
  fig.update_layout(xaxis_tickangle=-45, font=dict(size=15),
                    yaxis={'visible': False, 'showticklabels': False},                    
                    )  
  fig.update_traces(showlegend=False)

  fig.show();

interact(displayCelebrities, Top=widgets.IntSlider(min=0, max=30, step=1, value=10));

interactive(children=(IntSlider(value=10, description='Top', max=30), Output()), _dom_classes=('widget-interac…